In [262]:
import pandas as pd
import numpy as np
import json
import ast


In [263]:
messari = pd.read_csv('../data/messari.csv', index_col=0)
cg = pd.read_csv('../data/cg.csv', index_col=0)

In [264]:
messari.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100 entries, 0 to 99
Data columns (total 9 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   id_ms                       100 non-null    object
 1   serial_id_ms                100 non-null    int64 
 2   symbol_ms                   100 non-null    object
 3   name_ms                     100 non-null    object
 4   slug_ms                     100 non-null    object
 5   contract_addresses_ms       55 non-null     object
 6   _internal_temp_agora_id_ms  94 non-null     object
 7   metrics_ms                  100 non-null    object
 8   profile_ms                  100 non-null    object
dtypes: int64(1), object(8)
memory usage: 7.8+ KB


In [265]:
cg.rename(columns={"symbol_x_cg": "symbol_ms", "name_x_cg": "name_ms"}, inplace=True)


In [266]:
cg.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100 entries, 0 to 99
Data columns (total 59 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   id_cg                                100 non-null    object 
 1   symbol_ms                            100 non-null    object 
 2   name_ms                              100 non-null    object 
 3   image_x_cg                           100 non-null    object 
 4   current_price_cg                     100 non-null    float64
 5   market_cap_cg                        100 non-null    int64  
 6   market_cap_rank_x_cg                 100 non-null    int64  
 7   fully_diluted_valuation_cg           59 non-null     float64
 8   total_volume_cg                      100 non-null    float64
 9   high_24h_cg                          100 non-null    float64
 10  low_24h_cg                           100 non-null    float64
 11  price_change_24h_cg              

In [267]:
df = messari.merge(cg, how='outer', on='name_ms')

In [268]:
profile = df.iloc[0]

In [269]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 129 entries, 0 to 128
Data columns (total 67 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   id_ms                                100 non-null    object 
 1   serial_id_ms                         100 non-null    float64
 2   symbol_ms_x                          100 non-null    object 
 3   name_ms                              129 non-null    object 
 4   slug_ms                              100 non-null    object 
 5   contract_addresses_ms                55 non-null     object 
 6   _internal_temp_agora_id_ms           94 non-null     object 
 7   metrics_ms                           100 non-null    object 
 8   profile_ms                           100 non-null    object 
 9   id_cg                                100 non-null    object 
 10  symbol_ms_y                          100 non-null    object 
 11  image_x_cg                      

In [270]:
pd.json_normalize(ast.literal_eval(df.iloc[7]['market_data_cg']), max_level=0)

,current_price,total_value_locked,mcap_to_tvl_ratio,fdv_to_tvl_ratio,roi,ath,ath_change_percentage,ath_date,atl,atl_change_percentage,...,price_change_percentage_30d_in_currency,price_change_percentage_60d_in_currency,price_change_percentage_200d_in_currency,price_change_percentage_1y_in_currency,market_cap_change_24h_in_currency,market_cap_change_percentage_24h_in_currency,total_supply,max_supply,circulating_supply,last_updated
0,"{'aed': 1.8, 'ars': 63.43, 'aud': 0.71301, 'bc...",None,None,None,None,"{'aed': 11.34, 'ars': 301.61, 'aud': 4.19, 'bc...","{'aed': -84.08717, 'ars': -78.92901, 'aud': -8...","{'aed': '2021-09-02T06:00:10.474Z', 'ars': '20...","{'aed': 0.070719, 'ars': 0.373473, 'aud': 0.02...","{'aed': 2451.26961, 'ars': 16916.70238, 'aud':...",...,"{'aed': -0.51926, 'ars': 4.76443, 'aud': 0.881...","{'aed': -7.4171, 'ars': 1.1378, 'aud': -5.2430...","{'aed': -64.43578, 'ars': -55.18955, 'aud': -6...","{'aed': -53.75773, 'ars': -37.88377, 'aud': -5...","{'aed': -5326522230.136436, 'ars': -1834421971...","{'aed': -8.03059, 'ars': -7.86575, 'aud': -7.3...",4.500000e+10,4.500000e+10,3.382026e+10,2022-07-21T10:24:57.713Z


In [271]:
market_data = []

#TODO community_data, developer_data, metrics

for index, row in df.iterrows():
    try: 
        data = ast.literal_eval(row['market_data_cg'])
    except ValueError:
        data = {}
    data['name_ms'] = row['name_ms']
    market_data.append(pd.json_normalize(data, max_level=0).iloc[0].to_dict())
market_data = pd.DataFrame(market_data)

In [272]:
metrics_data = []

#TODO community_data, developer_data, metrics

for index, row in df.iterrows():
    try:
        metrics = ast.literal_eval(row['metrics_ms'])
    except ValueError:
        metrics = {}
    metrics['name_ms'] = row['name_ms']
    metrics_data.append(pd.json_normalize(metrics, max_level=0).iloc[0].to_dict())
metrics_data = pd.DataFrame(metrics_data)

In [273]:
community_data = []

#TODO community_data, developer_data, metrics

for index, row in df.iterrows():
    try:
        data = ast.literal_eval(row['community_data_cg'])
    except ValueError:
        data = {}
    data['name_ms'] = row['name_ms']
    community_data.append(pd.json_normalize(data, max_level=0).iloc[0].to_dict())
community_data = pd.DataFrame(community_data)

In [274]:
profile_data = []

#TODO community_data, developer_data, metrics

for index, row in df.iterrows():
    try:
        profile = ast.literal_eval(row['profile_ms'])
    except ValueError:
        profile = {}
    profile['name_ms'] = row['name_ms']
    profile_data.append(pd.json_normalize(profile, max_level=0).iloc[0].to_dict())

In [275]:
profile_data = pd.DataFrame(profile_data)

In [276]:
df = df.merge(profile_data, how='outer', on='name_ms')
df = df.merge(market_data, how='outer', on='name_ms')
df = df.merge(metrics_data, how='outer', on='name_ms')
df = df.merge(community_data, how='outer', on='name_ms')

In [277]:
general_data = []

#TODO community_data, developer_data, metrics

for index, row in df.iterrows():
    try:
        general = ast.literal_eval(str(row['general']))
    except ValueError:
        general = {}
    general['name_ms'] = row['name_ms']
    print(row['name_ms'])
    general_data.append(pd.json_normalize(general).iloc[0].to_dict())
general_data = pd.DataFrame(general_data)

Bitcoin
Ethereum
Tether
USD Coin
BNB
Binance USD
XRP
Cardano
Solana
Dogecoin
Polkadot
Dai
Polygon
Avalanche
Shiba Inu
TRON
Wrapped Bitcoin
Uniswap
Unus Sed Leo
Litecoin
FTX Token
Cronos
Ethereum Classic
Chainlink
NEAR Protocol
Cosmos
Stellar
Monero
Algorand
Bitcoin Cash
Flow
VeChain
The Sandbox
Decentraland
Internet Computer
Hedera Hashgraph
Tezos
FRAX
Filecoin
Aave
Axie Infinity
Elrond
Theta Network
TrueUSD
Quant Network
Helium
Bitcoin SV
EOS
KuCoin
Pax Dollar
Maker
THORChain
Zcash
OKB
IOTA
Fantom
Neutrino Dollar
Huobi Token
Klaytn
The Graph
NEO
Terra Luna Classic
Chiliz
Curve
Waves
Basic Attention Token
DeFiChain
PAX Gold
Stacks
Loopring
BitTorrent [OLD]
Enjin Coin
Zilliqa
Dash
Kusama
PancakeSwap
Arweave
Lido DAO
Convex Finance
NEM
Mina
Celo
Kava
Compound
Amp
Nexo
Gnosis
1inch
Gala
Holo
XDC Network
Decred
IoTeX
Synthetix Network Token
Bitcoin Gold
Qtum
Kadena
Theta Fuel
Jupiter
Harmony
Lido Staked Ether
LEO Token
FTX
Cosmos Hub
Chain
ApeCoin
Hedera
Frax
Quant
cUSDC
Huobi BTC
BitTorre

In [278]:
general_data

,roadmap,name_ms,overview.is_verified,overview.tagline,overview.category,overview.sector,overview.tags,overview.project_details,overview.official_links,background.background_details,background.issuing_organizations,regulation.regulatory_details,regulation.sfar_score,regulation.sfar_summary
0,"[{'title': 'Whitepaper', 'date': '2008-10-31T0...",Bitcoin,False,A peer-to-peer electronic cash system,Payments,Currencies,,"Bitcoin is the first <a href=""https://messari....","[{'name': 'Whitepaper', 'link': 'https://bitco...",\nPre-history\nBitcoin borrows from nearly thr...,[],"On April 28, 2018, the SEC Chairman, Jay Clayt...",1.0,Absence of a marketed token sale and correspon...
1,"[{'title': 'Olympic', 'date': '2015-05-01T04:0...",Ethereum,False,A decentralized computing platform,Infrastructure,Smart Contract Platforms,None,Ethereum is a distributed blockchain computing...,"[{'name': 'Whitepaper', 'link': 'https://ether...","Conception to token sale\n<a href=""https://mes...","[{'slug': 'ethereum-foundation', 'name': 'Ethe...","On June 14, 2018, the U.S. SEC's Director of C...",2.0,Current functionality of the platform\n\nAbsen...
2,[],Tether,False,Cryptocurrency backed by U.S. dollars,Payments,Stablecoins,,"Tether is fiat-collateralized <a href=""https:/...","[{'name': 'Website', 'link': 'https://tether.t...","Founded in July 2014 by <a href=""https://messa...","[{'slug': 'tether-limited', 'name': 'Tether Li...","Tether, along with parent company iFinex and r...",NaN,None
3,[],USD Coin,False,A USD stablecoin by Coinbase and Circle,Payments,Stablecoins,,USD Coin (USDC) is fiat-collateralized <a href...,"[{'name': 'Whitepaper', 'link': 'https://white...","Launched in October 2018, USD Coin (USDC) is a...","[{'slug': 'circle', 'name': 'Circle', 'logo': ...",None,NaN,None
4,"[{'title': 'Binance DEX Testnet', 'date': '201...",BNB,False,An exchange token native to the Binance blockc...,Financial,Smart Contract Platforms,"BSC Ecosystem, Cosmos Ecosystem",Binance Coin is digital asset native to the Bi...,"[{'name': 'Website', 'link': 'https://www.bina...",Binance is a cryptocurrency exchange that laun...,[],The token burn could be compared to a corporat...,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124,NaN,BitDAO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
125,NaN,ECOMI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
126,NaN,Curve DAO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
127,NaN,Frax Share,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [279]:
df = df.merge(general_data, how='outer', on='name_ms')

In [280]:
print(df.columns.tolist())


['id_ms', 'serial_id_ms', 'symbol_ms_x', 'name_ms', 'slug_ms', 'contract_addresses_ms', '_internal_temp_agora_id_ms', 'metrics_ms', 'profile_ms', 'id_cg', 'symbol_ms_y', 'image_x_cg', 'current_price_cg', 'market_cap_cg', 'market_cap_rank_x_cg', 'fully_diluted_valuation_cg', 'total_volume_cg', 'high_24h_cg', 'low_24h_cg', 'price_change_24h_cg', 'price_change_percentage_24h_cg', 'market_cap_change_24h_cg', 'market_cap_change_percentage_24h_cg', 'circulating_supply_cg', 'total_supply_cg', 'max_supply_cg', 'ath_cg', 'ath_change_percentage_cg', 'ath_date_cg', 'atl_cg', 'atl_change_percentage_cg', 'atl_date_cg', 'roi_cg', 'last_updated_x_cg', 'symbol_y_cg', 'name_y_cg', 'asset_platform_id_cg', 'platforms_cg', 'block_time_in_minutes_cg', 'hashing_algorithm_cg', 'categories_cg', 'public_notice_cg', 'additional_notices_cg', 'localization_cg', 'description_cg', 'links_cg', 'image_y_cg', 'country_origin_cg', 'genesis_date_cg', 'sentiment_votes_up_percentage_cg', 'sentiment_votes_down_percentage_c

In [281]:
df.drop(['general','market_data_cg', 'metrics_ms', 'community_data_cg', 'profile_ms', 'id_ms', 'serial_id_ms', '_internal_temp_agora_id_ms', 'public_notice_cg', 'additional_notices_cg', 'localization_cg', 'country_origin_cg', 'status_updates_cg'], axis=1, inplace=True)

In [282]:
df.to_csv('../data/coin_data.csv')